# Install tools via venv

```
cd missense_kinase_toolkit/schema
python3.11 -m venv VE
source VE/bin/activate

pip install -e .
pip install ipython
pip install jupyter
cd ../../notebooks

jupyter-lab
```

# Load packages

In [1]:
from missense_kinase_toolkit.schema import io_utils
import json

# Load data

In [2]:
dict_kinase = io_utils.deserialize_kinase_dict(deserialization_function=json.load)

# Examine `KinaseInfo` object

In [3]:
dir(dict_kinase["ABL1"])

['KLIFS',
 'KLIFS2UniProtIdx',
 'KLIFS2UniProtSeq',
 'KinCore',
 'KinHub',
 'Pfam',
 'UniProt',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydanti

In [4]:
dict_kinase["ABL1"].uniprot_id

'P00519'

In [5]:
dict_kinase["ABL1"].hgnc_name

'ABL1'

In [6]:
dict_kinase["ABL1"].KLIFS

KLIFS(gene_name='ABL1', name='ABL1', full_name='ABL proto-oncogene 1, non-receptor tyrosine kinase', group=<Group.TK: 'TK'>, family=<Family.Other: 'Other'>, iuphar=1923, kinase_id=392, pocket_seq='HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITEFMTYGNLLDYLREYLEKKNFIHRDLAARNCLVVADFGLS')

In [7]:
dict_kinase["ABL1"].KinCore

KinCore(seq='ITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFET', start=242, end=495, mismatch=None)

In [8]:
dict_kinase["ABL1"].KinHub

KinHub(kinase_name='Tyrosine-protein kinase ABL1', manning_name=['ABL'], xname=['ABL1'], group=[<Group.TK: 'TK'>], family=[<Family.Other: 'Other'>])

In [9]:
dict_kinase["ABL1"].UniProt

UniProt(canonical_seq='MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPIS

In [10]:
dict_kinase["ABL1"].KLIFS2UniProtIdx

{'I:1': 246,
 'I:2': 247,
 'I:3': 248,
 'g.l:4': 249,
 'g.l:5': 250,
 'g.l:6': 251,
 'g.l:7': 252,
 'g.l:8': 253,
 'g.l:9': 254,
 'II:10': 255,
 'II:11': 256,
 'II:12': 257,
 'II:13': 258,
 'III:14': 268,
 'III:15': 269,
 'III:16': 270,
 'III:17': 271,
 'III:18': 272,
 'III:19': 273,
 'αC:20': 282,
 'αC:21': 283,
 'αC:22': 284,
 'αC:23': 285,
 'αC:24': 286,
 'αC:25': 287,
 'αC:26': 288,
 'αC:27': 289,
 'αC:28': 290,
 'αC:29': 291,
 'αC:30': 292,
 'b.l:31': 293,
 'b.l:32': 294,
 'b.l:33': 296,
 'b.l:34': 297,
 'b.l:35': 298,
 'b.l:36': 299,
 'b.l:37': 300,
 'IV:38': 301,
 'IV:39': 302,
 'IV:40': 303,
 'IV:41': 304,
 'V:42': 312,
 'V:43': 313,
 'V:44': 314,
 'GK:45': 315,
 'hinge:46': 316,
 'hinge:47': 317,
 'hinge:48': 318,
 'linker:49': 319,
 'linker:50': 320,
 'linker:51': 321,
 'linker:52': 322,
 'αD:53': 323,
 'αD:54': 324,
 'αD:55': 325,
 'αD:56': 326,
 'αD:57': 327,
 'αD:58': 328,
 'αD:59': 329,
 'αE:60': 353,
 'αE:61': 354,
 'αE:62': 355,
 'αE:63': 356,
 'αE:64': 357,
 'VI:65': 3

In [11]:
dict_kinase["ABL1"].KLIFS2UniProtSeq

{'I': 'HKL',
 'g.l': 'GGGQYG',
 'II': 'EVYE',
 'II:III': 'GVWKKYSLT',
 'III': 'VAVKTL',
 'III:αC': 'KEDTMEVE',
 'αC': 'EFLKEAAVMKE',
 'b.l_1': 'IK',
 'b.l_intra': 'H',
 'b.l_2': 'PNLVQ',
 'IV': 'LLGV',
 'IV:V': 'CTREPPF',
 'V': 'YII',
 'GK': 'T',
 'hinge': 'EFM',
 'hinge:linker': None,
 'linker_1': 'T',
 'linker_intra': None,
 'linker_2': 'YGN',
 'αD': 'LLDYLRE',
 'αD:αE': 'CNRQEVNAVVLLYMATQISSAME',
 'αE': 'YLEKK',
 'αE:VI': None,
 'VI': 'NFI',
 'c.l': 'HRDLAARN',
 'VII': 'CLV',
 'VII:VIII': 'GENHLVK',
 'VIII': 'V',
 'xDFG': 'ADFG',
 'a.l': 'LS'}